In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import sent_tokenize
nltk.download('stopwords')
nltk.download('punkt')
import re

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Acer\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Acer\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
df = pd.read_csv("dataset/tennis.csv")
df.head()

,Unnamed: 0,article_id,article_text,source
0,0,1,Maria Sharapova has basically no friends as te...,https://www.tennisworldusa.org/tennis/news/Mar...
1,1,2,"BASEL, Switzerland (AP), Roger Federer advance...",http://www.tennis.com/pro-game/2018/10/copil-s...
2,2,3,Roger Federer has revealed that organisers of ...,https://scroll.in/field/899938/tennis-roger-fe...
3,3,4,Kei Nishikori will try to end his long losing ...,http://www.tennis.com/pro-game/2018/10/nishiko...
4,4,5,"Federer, 37, first broke through on tour over ...",https://www.express.co.uk/sport/tennis/1036101...


In [3]:
sentence = []
for i in df['article_text']:
    sentence.append(sent_tokenize(i))
sentences = [y for x in sentence for y in x]
print(sentences[0])

Maria Sharapova has basically no friends as tennis players on the WTA Tour.


In [7]:
wordembedd = {}
f = open('dataset/glove.6B.100d.txt', encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    wordembedd[word] = coefs
f.close()

In [8]:
cleansentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")
cleansentences = [s.lower() for s in cleansentences]
stop_words = stopwords.words('english')
len(stop_words)

179

In [9]:
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new
remove_stopwords("Maria Sharapova has basically no friends as tennis players on the WTA Tour.".split())

'Maria Sharapova basically friends tennis players WTA Tour.'

In [10]:
clean_sentences = [remove_stopwords(r.split()) for r in cleansentences]

In [11]:
sentence_vectors = []
for i in clean_sentences:
  if len(i) != 0:
    temp = sum([wordembedd.get(w, np.zeros((100,))) for w in i.split()])/(len(i.split())+0.001)
  else:
    temp = np.zeros((100,))
  sentence_vectors.append(temp)

In [12]:
matric = np.zeros([len(sentences), len(sentences)])
from sklearn.metrics.pairwise import cosine_similarity
for i in range(len(sentences)):
  for j in range(len(sentences)):
    if i != j:
      matric[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,100), sentence_vectors[j].reshape(1,100))[0,0]

In [13]:
import networkx as nx
nx_graph = nx.from_numpy_array(matric)
scores = nx.pagerank(nx_graph)

In [15]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
for i in range(1):
  print("Article:")
  print(df['article_text'][i])
  print('\n')
  print("Summary:")
  print(ranked_sentences[i][1])
  print('\n')

Article:
Maria Sharapova has basically no friends as tennis players on the WTA Tour. The Russian player has no problems in openly speaking about it and in a recent interview she said: 'I don't really hide any feelings too much. I think everyone knows this is my job here. When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match. I'm a pretty competitive girl. I say my hellos, but I'm not sending any players flowers as well. Uhm, I'm not really friendly or close to many players. I have not a lot of friends away from the courts.' When she said she is not really close to a lot of players, is that something strategic that she is doing? Is it different on the men's tour than the women's tour? 'No, not at all. I think just because you're in t

In [17]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
for i in range(1,2):
  print("Article:")
  print(df['article_text'][i])
  print('\n')
  print("Summary:")
  print(ranked_sentences[i][1])
  print('\n')

Article:
BASEL, Switzerland (AP), Roger Federer advanced to the 14th Swiss Indoors final of his career by beating seventh-seeded Daniil Medvedev 6-1, 6-4 on Saturday. Seeking a ninth title at his hometown event, and a 99th overall, Federer will play 93th-ranked Marius Copil on Sunday. Federer dominated the 20th-ranked Medvedev and had his first match-point chance to break serve again at 5-1. He then dropped his serve to love, and let another match point slip in Medvedev's next service game by netting a backhand. He clinched on his fourth chance when Medvedev netted from the baseline. Copil upset expectations of a Federer final against Alexander Zverev in a 6-3, 6-7 (6), 6-4 win over the fifth-ranked German in the earlier semifinal. The Romanian aims for a first title after arriving at Basel without a career win over a top-10 opponent. Copil has two after also beating No. 6 Marin Cilic in the second round. Copil fired 26 aces past Zverev and never dropped serve, clinching after 2 1/2 ho